<a href="https://colab.research.google.com/github/Wason1/Multum/blob/main/Multum_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multum

# Multum Overview
## What does the Multum package Change?  
- Adding new drugs to the reference table (that we might choose to add to our catalog) 
- Drug interaction checking 
- Dose range checking (not turned on) 
- Allergy Checking 
- List of all Drugs that we can add to the system (we can clearly see new ones that we don't have in our catalog) 

 
## Keys and Functionality
The CNUMs and DNUMs attached to an item are the Multum Keys that identify the item for functionality. 

CNUM: Uniquely identifies a synonyn. CNUM can be used for dose range checking. Paracetemol 500mg tablet or Paracetemol 665mg tablet are seperate CNUM's.  

DNUM: Identifies at primary level only. Eg paracetemol vs asprin. Used for interaction checking and allergy checking, but not dose range checking.  

 
CNUM ==SYNONYM_CKI== order_catalog_synonym.CKI 

DNUM == PRIMARY_CKI == order_catalog.CKI

# Create a folder
1. Make a folder on your c drive called 'MULTUM'
2. Copy this file (Multum.ipynb) into the 'MULTUM' folder on your c drive and open the copied ipynb file to continue working.
3. Create a sub folder called 'Cerner Multum Package'

# Download the Multum Package
Download the multum package into the 'MULTUM' folder. Put it in the 'Cerner Multum Package' folder then extract it in the same location

# Install The following Packages

- Snomed
- MBS (Medicare Benefits Schedule (AU))
- Multum (in cert, test it, then in production)

# Multum Order Catalogue Content Processing
# Duplicate Synonym Names
## run the two cells below

In [16]:
# This code is a function for copying code to the clipboard

from IPython.display import display, Javascript
import clipboard

def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)


code_block = """
# Your code here
"""
copy_code_block_to_clipboard(code_block)


<IPython.core.display.Javascript object>

# 3.1.	Duplicate Synonym Names (Multum 3)
Run the cell below to copy the CCL code and then run it in production.
paste the cutput into the spreadsheet and fix the duplicates by:
- zzz'd ing one
- other

In [17]:
# This code is a function for copying code to the clipboard
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
code_block = """
select 	; duplicate synonym mnemonic audit	; 'distinct' used for multiple virtual view joins on ocs_facility_r table and multiple orders on the orders table.
	catalog_type = cv_cat.display	
	, activity_type = cv_act.display	
	, activity_subtype = cv_sub.display	
	, primary_cki = oc.cki	
	, orderable_type = evaluate (oc.orderable_type_flag	
	, 0, "Standard"	
	, 1, "Standard"	
	, 2, "Supergroup"	
	, 3, "CarePlan"	
	, 4, "AP Special"	
	, 5, "Department Only"	
	, 6, "Order Set"	
	, 7, "Home Health Problem"	
	, 8, "Multi-ingredient"	
	, 9, "Interval Test"	
	, 10, "Freetext"	
	)	
	, primary_mnemonic = oc.primary_mnemonic	
	, synonym_cki = ocs.cki	
 	, synonym_type = uar_get_code_display(ocs.mnemonic_type_cd )	
 	, synonym_mnemonic = ocs.mnemonic	
	, synonym_active = ocs.active_ind	
	, synonym_oef = oef.oe_format_name	
	, synonym_hide = ocs.hide_flag	
	, synonym_last_update = format(ocs.updt_dt_tm, "dd/mm/yyyy hh:mm:ss")	
	, synonym_last_updater = if(ocs.synonym_id > 0 and ocs.updt_id = 0) "0"	
	else p_ocs.name_full_formatted	
	endif	
	, synonym_id = ocs.synonym_id	
		
 	, times_ordered = nullval(o.o_ttl,"0") ; "excluded from audit due to time-out"	; orders may need to excluded from PRDD2
	, fac.vv_facility 	
		
		
from		
	order_catalog_synonym ocs	
	, (left join prsnl p_ocs on p_ocs.person_id = ocs.updt_id)	; synonym last updater
	, (left join order_catalog oc on oc.catalog_cd = ocs.catalog_cd	
	and oc.orderable_type_flag not in (6,8)	
	)	
	, (left join order_entry_format oef on oef.oe_format_id = ocs.oe_format_id	
	and oef.action_type_cd = 2534	
	)	
	, (left join long_text l on l.long_text_id = ocs.high_alert_long_text_id)	; alert text
	, (left join code_value cv_cat on cv_cat.code_value = oc.catalog_type_cd)	
	, (left join code_value cv_act on cv_act.code_value = oc.activity_type_cd)	
	, (left join code_value cv_sub on cv_sub.code_value = oc.activity_subtype_cd)	
		
		
	, (left join 	
		
	(select fac.synonym_id, 	
	vv_facility = listagg(fac_cv.display,", ") over (order by fac_cv.display)	
	from ocs_facility_r fac	
	, (left join code_value fac_cv on fac_cv.code_value = fac.facility_cd)	
	group by fac.synonym_id)	
	fac on fac.synonym_id = ocs.synonym_id 	; synonym virtual view table
;	and fac.facility_cd = 0	; only return 'All Facilities' joins
	)	
		
		
	, (inner join order_catalog_synonym ocs_dup on ocs_dup.mnemonic_key_cap = ocs.mnemonic_key_cap	
	and ocs_dup.synonym_id != ocs.synonym_id	
	and (ocs_dup.cki = null or ocs.cki = null)	; ignore if duplicates are both Multum synonyms.
	and (ocs_dup.catalog_type_cd != 2519 or ocs.catalog_type_cd != 2519)	; ignore if duplicates are both surgery synonyms
	)	
	, (left join (select o_cnt.synonym_id, O_ttl = count(*) from orders o_cnt	
	group by o_cnt.synonym_id) o on o.synonym_id = ocs.synonym_id)	
		
plan	ocs	
		
;where 	ocs.catalog_type_cd not in (2513,2515,2516,2517)	; code values for 'Laborotory', 'Patient Care', 'Pharmacy' and 'Radiology' from code set 6000
;where 	ocs.catalog_type_cd = 2513 	; code value for 'Laboratory' from code set 6000
;where 	ocs.catalog_type_cd = 2515	; code value for 'Patient Care' from code set 6000
;where 	ocs.catalog_type_cd = 2516	; code value for 'Pharmacy' from code set 6000
;where 	ocs.catalog_type_cd = 2517 	; code value for 'Radiology' from code set 6000
		
join	p_ocs	
join	oc	
join 	oef	
join	l	
join	cv_cat	
join	cv_act	
join	cv_sub	
join	fac	
join	ocs_dup	
join	o	
		
order by		
	ocs.mnemonic_key_cap	
	, ocs.active_ind desc	; show active synonyms on top
;	, cv_cat.display_key	; corrupts the 'select distinct' if included where only one instance of some duplicate synonyms are displayed.
;	, cv_act.display_key	; corrupts the 'select distinct' if included where only one instance of some duplicate synonyms are displayed.
;	, cv_sub.display_key	; for some reason, corrupts the 'select distinct' if included.
	, cnvtupper(oc.primary_mnemonic)	
	, evaluate(ocs.mnemonic_type_cd	; synonym_type custom list with "Primary" first, as per DCP tools.
	, 2583, 01	; "Primary"
	, 2579, 02	; "Ancillary"
	, 2580, 03	; "Brand Name"
	, 614542, 04	; "C - Dispensable Drug Names"
	, 2581, 05	; "Direct Care Provider"
	, 614543, 06	; "E - IV Fluids and Nicknames"
	, 2582, 07	; "Generic Name"
	, 614544, 08	; "M - Generic Miscellaneous Products"
	, 614545, 09	; "N - Trade Miscellaneous Products"
	, 614546, 10	; "Outreach"
	, 614547, 11	; "PathLink"
	, 2584, 12	; "Rx Mnemonic"
	, 2585, 13	; "Surgery Med"
	, 614548, 14	; "Y - Generic Products"
	, 614549, 15	; "Z - Trade Products"
	)	
	, ocs.synonym_id	
	, 0	; in case 'select distinct' is used
		
with	time = 1200	
"""
copy_code_block_to_clipboard(code_block)


<IPython.core.display.Javascript object>

# 3.2 Inactive Synonyms Used In Active Inpatient Orders (Multum 4)
Run the cell below to copy the CCL code and then run it in production. These items should not be in active orders because they won't work (they're inactive). If the query gets hits then we need to find the owners of the items (synonyms) and orders then paste the cutput into the spreadsheet.

 Possible solutions include:
- removing the inactive item (synonym) from the order
- other

In [21]:
# This code is a function for copying code to the clipboard
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
code_block = """
select	; inactive synonym in currently active orders	
	UR_number = ea_URN.alias	
	, patient_name = p.name_full_formatted  ;"xxxx"	
	, patient_id = o.person_id	
	, encntr_dates = concat(format(e_orig.arrive_dt_tm, "dd/mm/yy hh:mm"), " - ", format(e_orig.depart_dt_tm, "dd/mm/yy hh:mm"))	
	, visit_no = ea_visit.alias	
	, o.encntr_id	
	, facility_at_time_of_order = uar_get_code_display(e_orig.loc_facility_cd)	
	, unit_at_time_of_order = if(elh.loc_nurse_unit_cd > 0) uar_get_code_display(elh.loc_nurse_unit_cd)	
	else uar_get_code_display(e_orig.loc_nurse_unit_cd)	
	endif	
;	, current_facility = if(e_curr.encntr_id > 0) uar_get_code_display(e_curr.loc_facility_cd)	
;	else "no current facility"	
;	endif	
;	, current_unit = if(e_curr.encntr_id > 0) uar_get_code_display(e_curr.loc_nurse_unit_cd)	
;	else "no current unit"	
;	endif	
	, order_type = evaluate (o.orig_ord_as_flag,	
	0, "Normal Order",	
	1, "Prescription/Discharge Order",	
	2, "Recorded / Home Meds",	
	3, "Patient Owns Meds",	
	4, "Pharmacy Charge Only",	
	5, "Satellite (Super Bill) Meds"	
	)	
;	, med_order_type = uar_get_code_display(o.med_order_type_cd)	
	, original_order_date = format(o.orig_order_dt_tm, "dd/mm/yyyy hh:mm:ss")	
	, order_placed_by = p_o_a_order.name_full_formatted	
	, order_projected_stop_date = format(o.projected_stop_dt_tm, "dd/mm/yyyy hh:mm:ss")	
	, current_order_status = uar_get_code_display(o.order_status_cd)	
	, order_status_last_update = format(o.status_dt_tm, "dd/mm/yyyy hh:mm:ss")	
	, order_status_last_updater = if(o.order_id > 0 and o.status_prsnl_id = 0) "0"	
	else p_o_stat.name_full_formatted	
	endif	
	, order_last_update = format(o.updt_dt_tm, "dd/mm/yyyy hh:mm:ss")	
	, order_last_updater = if(o.order_id > 0 and o.updt_id = 0) "0"	
	else p_o.name_full_formatted	
	endif	
	, o.clinical_display_line	
	, o.order_id	
	, o.ordered_as_mnemonic	
	, synonym_mnemonic = ocs.mnemonic	
	, synonym_type = uar_get_code_display(ocs.mnemonic_type_cd)	
	, synonym_id = o.synonym_id	
	, order_rank = dense_rank() over (partition by 0	
	order by	
	o.orig_order_dt_tm	
	, o.status_dt_tm	
	, o.order_id	
	)	
		
from		
	orders o	
	, (left join prsnl p_o on p_o.person_id = o.updt_id)	
	, (left join prsnl p_o_stat on p_o_stat.person_id = o.status_prsnl_id)	
	, (left join encounter e_orig on e_orig.encntr_id = o.encntr_id)	
	, (left join encntr_alias ea_URN on ea_URN.encntr_id = o.encntr_id	
	and ea_URN.encntr_alias_type_cd = 1079	; 'URN' from code set 319
	and ea_URN.active_ind = 1	; active URNs only
	and ea_URN.end_effective_dt_tm > sysdate	; effective URNs only
	)	
	, (left join encntr_alias ea_visit on ea_visit.encntr_id = o.encntr_id	
	and ea_visit.encntr_alias_type_cd = 1077	; 'FIN NBR' from code set 319
	and ea_visit.active_ind = 1	; active FIN NBRs only
	and ea_visit.end_effective_dt_tm > sysdate	; effective FIN NBRs only
	)	
	, (left join encntr_loc_hist elh on elh.encntr_id = o.encntr_id	
	and elh.active_ind = 1	; to remove inactive rows that seem to appear for unknown reason(s)
	and elh.pm_hist_tracking_id > 0	; to remove duplicate row that seems to occur at discharge
	and elh.beg_effective_dt_tm < o.orig_order_dt_tm	; encounter location began before order was placed
	and elh.end_effective_dt_tm >  o.orig_order_dt_tm	; encounter location ended after order was placed
	)	
	, (left join order_action o_a_order on o_a_order.order_id = o.order_id	
	 and o_a_order.action_type_cd = 2534	; 'order' from codeset 6003
	)	
	, (left join prsnl p_o_a_order on p_o_a_order.person_id = o_a_order.action_personnel_id)	
	, (left join person p on p.person_id = o.person_id)	
	, (left join order_catalog_synonym ocs on ocs.synonym_id = o.synonym_id)	
;	, (left join encounter e_curr on e_curr.person_id = o.person_id	
;	and e_curr.encntr_type_cd in (309308, 309310)	; 'Inpatient' and 'Emergency' from codeset 71
;	and e_curr.active_ind = 1	
;	and e_curr.arrive_dt_tm < sysdate	; patient arrived in the past
;	and e_curr.depart_dt_tm = null	; but has not yet departed
;	)	
plan	o	
where	o.template_order_id = 0	
;and	o.orig_order_dt_tm > cnvtdatetime("01-DEC-2020")	
and	o.order_status_cd in (	
	2546	; Future
	, 2547	; Incomplete
	, 2548	; InProcess
	, 2549	; On Hold, Med Student
	, 2550	; Ordered
	, 2551	; Pending Review
	, 2552	; Suspended
	, 2553	; Unscheduled
	, 614538	; Transfer/Canceled
	, 643466	; Pending Complete
	)	
and	o.orig_ord_as_flag = 0	; inpatient orders only
;and	o.orig_ord_as_flag = 1	; discharge prescriptions only
;and	o.orig_ord_as_flag = 2	; Recorded / Home Meds only
and	o.active_ind = 1	; active orders only
and	(o.projected_stop_dt_tm > sysdate	; current orders only (future stop date or no stop date)
	or	
	o.projected_stop_dt_tm = null	
	)	
and	o.synonym_id not in (select synonym_id	
	from order_catalog_synonym	
	where active_ind = 1	
	)	
join	p_o	
join	p_o_stat	
join	e_orig	
join	ea_URN	
join	ea_visit	
join	elh	
join	o_a_order	
;where	o_a_order.updt_id = 1235678	; orders placed by …
;and	o_a_order.updt_dt_tm between cnvtdatetime("01-DEC-2016") and cnvtdatetime("01-DEC-2020")	; between dates
join	p_o_a_order	
join	p	
join	ocs	
;join	e_curr	
order by		
	o.orig_order_dt_tm	
	, o.status_dt_tm	
	, o.order_id	
	, ea_URN.alias	
		
with	time = 1200	
"""
copy_code_block_to_clipboard(code_block)

<IPython.core.display.Javascript object>

# Multum 5 Inactive Synonyms Used In Care Sets, IVs And PowerPlans
Run the extract script in each Production environment and paste into the appropriate worksheets
- if no data comes back nothing needs to be fixed, move on.
- if data comes back then we need to investigate to figure out if we can remove the inactive synonyms from the powerforms

In [24]:
# Copy PowerPlan extract script (PowerPlans with inactive component synonyms)
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
code_block = """
select	; PowerPlans with inactive component synonyms	
	plan_display = p_cat.display_description	
	, plan_or_phase_description = p_cat.description	
	, plan_type = p_cat.type_mean	
	, plan_active_ind = p_cat.active_ind	
	, plan_version = p_cat.version	
	, plan_currency = if (p_cat.version = 0 or  p_cat.version = p_cat_curr_v.version) "current"	
	else "obsolete"	
	endif	
	, plan_status = if (p_cat.beg_effective_dt_tm > cnvtdatetime(curdate, curtime3) and p_cat.end_effective_dt_tm > cnvtdatetime(curdate, curtime3)) "testing"	
	elseif (p_cat.beg_effective_dt_tm < cnvtdatetime(curdate, curtime3) and p_cat.end_effective_dt_tm > cnvtdatetime(curdate, curtime3)) "production"	
	elseif (p_cat.end_effective_dt_tm < cnvtdatetime(curdate, curtime3)) "inactive"	
	endif	
	, comp_clinical_category = if (p_comp.pathway_comp_id > 0 and pw_os.order_sentence_seq < 2) uar_get_code_display(p_comp.dcp_clin_cat_cd)	; return if row is a component, and either non-order sentence, or first order sentence only
	else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	endif	
	, comp_clinical_subcategory = if (p_comp.pathway_comp_id > 0 and pw_os.order_sentence_seq < 2) uar_get_code_display(p_comp.dcp_clin_sub_cat_cd)	; return if row is a component, and either non-order sentence, or first order sentence only
	else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	endif	
	, comp_sequence = if (p_comp.pathway_comp_id > 0 and pw_os.order_sentence_seq < 2) cnvtstring(p_comp.sequence)	; return if row is a component, and either non-order sentence, or first order sentence only
	else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	endif	
	, comp_type = if (pw_os.order_sentence_seq < 2) 	; return if row is non-order sentence, or first order sentence row
	if (p_comp.pathway_catalog_id > 0) uar_get_code_display(p_comp.comp_type_cd)	
	elseif (p_rel.pw_cat_t_id > 0) "Phase"	
	endif	
	else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	endif	
	, comp_name = if (pw_os.order_sentence_seq < 2)	; return if row is non-order sentence, or first order sentence row
	if (p_comp.comp_type_cd = 10734) substring(1,500,l.long_text)	; "Note"
	elseif (p_comp.comp_type_cd in (10736, 7326766, 7057725)) ocs.mnemonic	; "Order", "Prescription" (*Note: different "Prescription" codes for G1 and G2)
	elseif (p_comp.comp_type_cd = 10738) occ.description	; "Result Outcome"
	elseif (p_comp.comp_type_cd = 3606218) p_cat_sub.description	; "Sub Phase"
	elseif (p_rel.pw_cat_t_id > 0) p_cat_phase.description	; "Phase"
	elseif (p_comp.pathway_comp_id = 0) ""	; Not a component
	else "-  - component type not configured in extract script - -"	
	endif	
	 else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	 endif 	
	, comp_active_in_system = if (pw_os.order_sentence_seq < 2)	; return if row is non-order sentence, or first order sentence row
	if (p_comp.comp_type_cd = 10734 and l.long_text_id = 0 ) "1"	; blank "Note"
	elseif (p_comp.comp_type_cd = 10734) evaluate (l.active_ind, 0, "0", 1, "1")	; "Note"
	elseif (p_comp.comp_type_cd in (10736, 7326766, 7057725)) evaluate (ocs.active_ind, 0, "0", 1, "1")	; "Order", "Prescription" (*Note: different "Prescription" codes for G1 /G5/G6 and G2/G3/G4)
	elseif (p_comp.comp_type_cd = 10738) evaluate (occ.active_ind, 0, "0",1 , "1")	; "Result Outcome"
	elseif (p_comp.comp_type_cd = 3606218) evaluate (p_cat_sub.active_ind, 0, "0", 1, "1")	; "Sub Phase"
	elseif (p_rel.pw_cat_t_id > 0) evaluate (p_cat_phase.active_ind, 0, "0", 1, "1")	; "Phase"
	elseif (p_comp.pathway_comp_id = 0) ""	; Not a component
	else "-  - component type not configured in extract script - -"	
	endif	
	 else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	 endif 	
	, comp_active_in_plan = if (p_comp.pathway_catalog_id > 0) evaluate (p_comp.active_ind, 0, "0", 1, "1")	
	elseif (p_rel.pw_cat_t_id > 0) "1"	
	else ""	
	endif	
	, comp_include = if (p_comp.pathway_comp_id > 0 and pw_os.order_sentence_seq < 2)	; return if row is non-order sentence, or first order sentence row
	if (p_comp.include_ind = 1) "1"	
	else "0"	
	endif	
	 else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	 endif 	
	, comp_required = if (p_comp.pathway_comp_id > 0 and pw_os.order_sentence_seq < 2)	
	if (p_comp.required_ind = 1) "1"	
	else "0"	
	endif	
	 else ""	
	 endif 	
	, comp_order_sentence = os.order_sentence_display_line	
	, comp_last_update = format(p_comp.updt_dt_tm, "dd/mm/yyyy hh:mm:ss")	
	, comp_last_updater = if(p_comp.pathway_comp_id > 0 and p_comp.updt_id = 0) "0"	
	else p_p_comp.name_full_formatted	
	endif	
	, plan_version_catalog_id = p_cat.pathway_catalog_id	
	, plan_all_versions_id = p_cat.version_pw_cat_id	
	, comp_id = if (p_comp.pathway_comp_id > 0 and pw_os.order_sentence_seq < 2)  cnvtstring(p_comp.pathway_comp_id)	; return if row is a component, but either non-order sentence, or first order sentence row
	elseif (p_rel.pw_cat_t_id > 0)  cnvtstring(p_rel.pw_cat_t_id)	; for components which are other powerplans ("Phase")
	else ""	; do not return duplicate information if row is non-order sentence, or not first order sentence row
	endif	
	, comp_os_id = if (pw_os.order_sentence_id > 0) cnvtstring(pw_os.order_sentence_id)	
	else ""	
	endif	
	, powerplan_rank = dense_rank() over (partition by 0	
	order by p_cat.description_key	
	, p_cat.version	
	)	
		
from		
	pathway_catalog p_cat	
	, pathway_catalog p_cat_curr_v 	
	, pathway_comp p_comp	
	, prsnl p_p_comp	
	, code_value clin_cat_cv	; for p_comp.dcp_clin_cat_cd sequence
	, long_text l	; note within a plan
	, order_catalog_synonym ocs	; synonym (orderable or presription) within a plan
	, outcome_catalog occ	
	, pathway_catalog p_cat_sub	; sub-phase within a care plan
	, pw_cat_reltn p_rel	; phase / pathway relationship
	, pathway_catalog p_cat_phase	; phase within pathway
	, pw_comp_os_reltn pw_os	
	, order_sentence os	
		
plan	p_cat	
where	p_cat.pathway_catalog_id > 0	; to remove dummy row from results
and	p_cat.pathway_catalog_id in (select pathway_catalog_id	; PowerPlan
	from pathway_comp	
	where active_ind = 1	
	and parent_entity_id in (select synonym_id	;containing a pathway component
	from order_catalog_synonym	
	where active_ind = 0	; that is inactive
	)	
	)	
;and	p_cat.description_key < "D*"	
;and	p_cat.description_key >= "D*"	
		
join	p_cat_curr_v	
where	p_cat_curr_v.version_pw_cat_id = p_cat.version_pw_cat_id	
and	(	
	p_cat_curr_v.version = (select max(version)	
	from pathway_catalog	
	where version_pw_cat_id = p_cat_curr_v.version_pw_cat_id	
	and p_cat.type_mean not in ("DOT", "PHASE")	
	)	
	or	
	p_cat_curr_v.pathway_catalog_id = (select max(pathway_catalog_id)	
	from pathway_catalog	
	where version_pw_cat_id = p_cat_curr_v.version_pw_cat_id	
	and p_cat.type_mean in ("DOT", "PHASE")	
	)	
	)	
		
join	p_comp	
where	p_comp.pathway_catalog_id = outerjoin(p_cat.pathway_catalog_id)	
		
join	p_p_comp	
where	p_p_comp.person_id = outerjoin(p_comp.updt_id)	
		
join	clin_cat_cv	
where	clin_cat_cv.code_value = outerjoin(p_comp.dcp_clin_cat_cd)	
		
join	l	
where	l.long_text_id = outerjoin(p_comp.parent_entity_id)	
and	l.parent_entity_name = outerjoin("PATHWAY_COMP")	
		
join	ocs	
where	ocs.synonym_id = outerjoin(p_comp.parent_entity_id)	
		
join	occ	
where	occ.outcome_catalog_id = outerjoin(p_comp.parent_entity_id)	
		
join	p_cat_sub	
where	p_cat_sub.pathway_catalog_id = outerjoin(p_comp.parent_entity_id)	
		
join	p_rel	
where	p_rel.pw_cat_s_id = outerjoin(p_cat.pathway_catalog_id)	
and	p_rel.type_mean = outerjoin("GROUP")	
		
join	p_cat_phase	
where	p_cat_phase.pathway_catalog_id = outerjoin(p_rel.pw_cat_t_id)	
		
join	pw_os	
where	pw_os.pathway_comp_id = outerjoin(p_comp.pathway_comp_id)	
		
join	os	
where	os.order_sentence_id = outerjoin(pw_os.order_sentence_id)	
		
order by		
	p_cat.description_key	
	, p_cat.version	
	, p_cat.pathway_catalog_id	
	, clin_cat_cv.collation_seq	
	, p_comp.sequence	
	, pw_os.order_sentence_seq	
		
with	time = 180	

"""
copy_code_block_to_clipboard(code_block)

<IPython.core.display.Javascript object>

In [26]:
# Care Sets and IVs with inactive component  synonyms extract script
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
code_block = """
select	; Care Sets and IVs with inactive component  synonyms	
	catalog_type = cv_cat.display	
	, activity_type = cv_act.display	
	, sub_activity_type = cv_sub.display	
	, careset_type = evaluate (oc.orderable_type_flag	
	, 0, "Standard"	
	, 1, "Standard"	
	, 2, "Supergroup"	
	, 3, "CarePlan"	
	, 4, "AP Special"	
	, 5, "Department Only"	
	, 6, "Order Set"	
	, 7, "Home Health Problem"	
	, 8, "Multi-ingredient"	
	, 9, "Interval Test"	
	, 10, "Freetext")	
	, careset_primary = oc.primary_mnemonic	
	, careset_active_ind = oc.active_ind	
	, comp_sequence = cs_comp.comp_seq	
	, comp_type = uar_get_code_display(cs_comp.comp_type_cd)	
	, component_name = if (cs_comp.comp_type_cd = 2716) cs_comp.comp_label	; label
	elseif (cs_comp.comp_type_cd = 2717) substring(1,500,l_comp.long_text)	; note
	elseif (cs_comp.comp_type_cd = 2718) ocs_comp.mnemonic	; orderable
	else "-  - component type not configured in extract script - -"	
	endif	
	, comp_active_in_system = if (cs_comp.comp_type_cd = 2716) ""	; label
	elseif (cs_comp.comp_type_cd = 2717) evaluate (l_comp.active_ind,0,"0",1,"1")	; note
	elseif (cs_comp.comp_type_cd = 2718) evaluate (ocs_comp.active_ind,0,"0",1,"1")	; orderable
	else "-  - component type not configured in extract script - -"	
	endif	
	, comp_inclusion = if (cs_comp.comp_type_cd in (2716, 2717)) ""	; label or note
	elseif (cs_comp.required_ind = 1) "Required"	
	elseif (cs_comp.include_exclude_ind = 1) "Include"	
	elseif (cs_comp.include_exclude_ind = 0) "Exclude"	
	endif	
	, comp_os = os.order_sentence_display_line	
	, comp_os_comment = substring(1,500,l_os_com.long_text)	
	, comp_last_update = format(cs_comp.updt_dt_tm, "dd/mm/yyyy hh:mm:ss")	
	, comp_last_updater = if(cs_comp.comp_id > 0 and cs_comp.updt_id = 0) "0"	
	else p_cs_comp.name_full_formatted	
	endif	
	, careset_catalog_cd = oc.catalog_cd	
	, comp_id = if (cs_comp.comp_type_cd = 2716) cs_comp.comp_label	; label - no code exists. Entire label is stored
	elseif (cs_comp.comp_type_cd = 2717) cnvtstring(cs_comp.long_text_id)	; note
	elseif (cs_comp.comp_type_cd = 2718) cnvtstring(cs_comp.comp_id)	; orderable
	else "-  - component type not configured in extract script - -"	
	endif	
	, comp_os_id = if (cs_comp.order_sentence_id > 0) cnvtstring(cs_comp.order_sentence_id)	
	else ""	
	endif	
	, careset_rank = dense_rank() over (partition by 0	; no logical database field partition
	order by cv_cat.display_key	
	, cv_act.display_key	
	, cv_sub.display_key	
	, cnvtupper(oc.primary_mnemonic)	
	)	
		
from 		
	order_catalog oc	
	, (left join code_value cv_cat on cv_cat.code_value = oc.catalog_type_cd)	
	, (left join code_value cv_act on cv_act.code_value = oc.activity_type_cd)	
	, (left join code_value cv_sub on cv_sub.code_value = oc.activity_subtype_cd)	
;	, (left join order_catalog_synonym ocs_cs on ocs_cs.catalog_cd = oc.catalog_cd	; synonym names for care set
;	 and ocs_cs.mnemonic_type_cd != 2583	; exclude Primaries
;	)	
	, (left join cs_component cs_comp on cs_comp.catalog_cd = oc.catalog_cd)	
	, (left join prsnl p_cs_comp on p_cs_comp.person_id = cs_comp.updt_id)	
	, (left join long_text l_comp on l_comp.long_text_id = cs_comp.long_text_id)	
	, (left join order_catalog_synonym ocs_comp on ocs_comp.synonym_id = cs_comp.comp_id)	
	, (left join order_sentence os on os.order_sentence_id = cs_comp.order_sentence_id)	
	, (left join long_text l_os_com on l_os_com.long_text_id = os.ord_comment_long_text_id)	
		
plan	oc	
where	oc.orderable_type_flag in (6,8)	; Order Set, Multi-ingredient (IV Set)
and	oc.catalog_cd in (select distinct catalog_cd	; Care Set / IV Set
	from cs_component	
	where comp_id in (select synonym_id	; containing a component
	from order_catalog_synonym	
	where active_ind = 0	; that is inactive
	)	
	)	
		
join	cv_cat	
join	cv_act	
join	cv_sub	
;join	ocs_cs	
join	cs_comp	
join	p_cs_comp	
join	l_comp	
join	ocs_comp	
join	os	
join	l_os_com	
		
order by		
	cv_cat.display_key	
	, cv_act.display_key	
	, cv_sub.display_key	
	, cnvtupper (oc.primary_mnemonic)	
	, cs_comp.comp_seq	
		
with	time = 60	

"""
copy_code_block_to_clipboard(code_block)

<IPython.core.display.Javascript object>

# Multum 6 Obsolete / Invalid CNUMs
- Open ‘Multum Processing 6 - catalogue-Multum_Alignment.xlsx’ found in Data Management Team\Multum and save a copy in the relevant Multum package folder.
- Run the ‘syn-Mul compare extract script’ in each Production environment and paste into the appropriate worksheets.
- Run the ‘current Multum extract script’ and in any Production environment and paste the results into the ‘current Multum’ worksheet.
- Run the ‘obsolete Multum extract script’ in any Production environment and paste into the ‘obsolete Multum’ worksheet.

In [28]:
# COPY syn-Mul compare extract script
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
code_block = """select	; synonym - Multum comparison	
	primary_DNUM = oc.cki	
	, multum_DNUM = if(mocl.catalog_cki != null) mocl.catalog_cki	; if mocl.catalog_cki exists, return mocl.catalog_cki
	elseif (mul_dn_map.drug_identifier != null) mul_dn_map.drug_identifier	; if mltmmdc.drug_identifier exists, return mltmmdc.drug_identifier
	else "-"	; if neither mocl.catalog_cki nor mltmmdc.drug_identifier, return "-"
	endif	
	, DNUM_match =  if (mocl.catalog_cki != null)	; if mocl.catalog_cki exists
	if (mocl.catalog_cki = oc.cki) "1"	; if mocl.catalog_cki exists and oc.cki matches mocl.catalog_cki, return "1"
	else "0"	; if mocl.catalog_cki exists but oc.cki does not match mocl.catalog_cki, return "0"
	endif	
	elseif (mul_dn_map.drug_identifier != null)	; if mltmmdc.drug_identifier exists
	if (findstring(mul_dn_map.drug_identifier,oc.cki) > 0) "1"	; if mltmmdc.drug_identifier exists and mltmmdc.drug_identifier is found within oc.cki, return "1"
	else "0"	; if mltmmdc.drug_identifier exists but mltmmdc.drug_identifier is not found within oc.cki, return "0"
	endif	
	elseif (oc.cki not in (null,"IGNORE") and ocs.mnemonic_type_cd = 2583) "0"	; if DNUM exists, and synonym is a primary, return "0"
	else "-"	; if neither mocl.catalog_cki nor mltmmdc.drug_identifier exist, return "-"
	endif	
	, ignore_DNUM_mapping = if (b_dnum_ig.br_name_value_id > 0) 1	
	else 0	
	endif	
	, primary_mnemonic = oc.primary_mnemonic	
	, primary_catalog_cd = oc.catalog_cd	
	, synonym_CNUM = ocs.cki	
	, ignore_CNUM_mapping = if (b_cnum_ig.br_name_value_id > 0) 1	
	else 0	
	endif	
	, CNUM_count = count(ocs.synonym_id) over (partition by ocs.cki)	
	, multum_syn_cki_status = if (ocs.cki in (""," ",null)) "-"	; if CNUM is not populated, return "-"
 	elseif (mul_dn.is_obsolete = "F") "current"	
 	elseif (mul_dn.is_obsolete = "T") "obsolete"	
 	elseif (mul_dn.drug_synonym_id = 0) "invalid"	; if CNUM does not create a join on mul_dn, return "invalid"
 	endif	
 	, synonym_type = uar_get_code_display(ocs.mnemonic_type_cd)	
	, multum_type = if (mocl.mnemonic_type != null) mocl.mnemonic_type	; if mocl.mnemonic_type exists, return mocl.mnemonic_type
	elseif (mul_dn_map.function_id != null) evaluate (mul_dn_map.function_id	; if mul_dn_map.function_id exists, evaluate mul_dn_map.function_id
	, 16, "Primary"	
	, 29, "Brand Drug Name"	
	, 17, "Brand Name"	
	, 26, "C - Dispensable Drug Names"	
	, 19, "Drug Nickname"	
	, 62, "E - IV Fluids and Nicknames"	
	, 61, "IV Fluid"	
	, 59, "M - Generic Miscellaneous Products/Y - Generic Products"	
	, 64, "Multivitamin Name"	
	, 60, "N - Trade Miscellaneous Products/Z - Trade Products"	
	, 0, "-synonym not found on mul_dn_map table-"	
	)	
 	else "-"	; if neither mocl.mnemonic_type nor mul_dn_map.function_id exists, return "-"
	endif	
 	, type_match = if (ocs.active_ind = 1 and mocl.mnemonic_type != null)	; if mocl.mnemonic_type exists
	if (uar_get_code_display(ocs.mnemonic_type_cd) = mocl.mnemonic_type) "1"	; if mocl.mnemonic_type exists and ocs.mnemonic_type matches mocl.mnemonic_type, return "1"
	else "0"	; if mocl.mnemonic_type exists but ocs.mnemonic_type does not match mocl.mnemonic_type, return "0"
	endif	
	elseif (ocs.active_ind = 1 and mul_dn_map.function_id != null)	; if mul_dn_map.function_id exists
	if(findstring(trim(uar_get_code_display(ocs.mnemonic_type_cd)), evaluate(mul_dn_map.function_id	; and 'ocs.mnemonic_type' string is found within the 'multum function' string
	, 16, "Primary"	
	, 29, "Brand Drug Name"	
	, 17, "Brand Name"	
	, 26, "C - Dispensable Drug Names"	
	, 19, "Drug Nickname"	
	, 62, "E - IV Fluids and Nicknames"	
	, 61, "IV Fluid"	
	, 59, "M - Generic Miscellaneous Products/Y - Generic Products"	
	, 64, "Multivitamin Name"	
	, 60, "N - Trade Miscellaneous Products/Z - Trade Products"	
	)	
	)	
	 > 0) "1"	; , return "1"
	else "0"	; if mul_dn_map.function_id exists but 'ocs.mnemonic_type' string is not found within the 'multum function' string, return "0"
	endif	
	else "-"	; if neither mocl.mnemonic_type nor mul_dn_map.function_id exists, return "-"
	endif	
 	, synonym_mnemonic = ocs.mnemonic	
	, duplicate_mnemonic_count = count(ocs.synonym_id) over (partition by ocs.mnemonic)	
 	, multum_mnemonic = if (mul_manf_dn.manufacturer_ordered_drug_name != null) mul_manf_dn.manufacturer_ordered_drug_name	; if mul_manf_dn.manufacturer_ordered_drug_name exists for this CNUM, return mul_manf_dn.manufacturer_ordered_drug_name
	elseif (mul_dn.drug_name != null) mul_dn.drug_name	; if mul_manf_dn.manufacturer_ordered_drug_name does not exist for this CNUM, but mul_dn.drug_name exists, return mul_dn.drug_name
	else "-"	; if neither mul_manf_dn.manufacturer_ordered_drug_name nor mul_dn.drug_name exist for this CNUM, return "-"
	endif	
 	, mnemonic_match = if (ocs.active_ind = 1 and mul_manf_dn.manufacturer_ordered_drug_name != null)	; if mul_manf_dn.manufacturer_ordered_drug_name exists for this CNUM
	if (textlen(mul_manf_dn.manufacturer_ordered_drug_name) > 100) "Multum mnemonic > 100 characters"	; if mul_manf_dn.manufacturer_ordered_drug_name exists for this CNUM but is greater than 100 characters, return "Multum mnemonic > 100 characters"
	elseif (ocs.mnemonic = mul_manf_dn.manufacturer_ordered_drug_name != null) "1"	; if mul_manf_dn.manufacturer_ordered_drug_name exists for this CNUM and ocs.mnemonic matches mul_manf_dn.manufacturer_ordered_drug_name, return "1"
	else "0"	; if mul_manf_dn.manufacturer_ordered_drug_name exists for this CNUM but ocs.mnemonic does not match mul_manf_dn.manufacturer_ordered_drug_name, return "0"
	endif	
	elseif  (ocs.active_ind = 1 and mul_dn.drug_name != null)	; if mul_dn.drug_name exists for this CNUM
	if (textlen(mul_dn.drug_name) > 100) "Multum mnemonic > 100 characters"	; if mul_dn.drug_name exists for this CNUM but is greater than 100 characters, return "Multum mnemonic > 100 characters"
	elseif (ocs.mnemonic = mul_dn.drug_name) "1"	; if mul_dn.drug_name exists for this CNUM and ocs.mnemonic matches mul_dn.drug_name, return "1"
	else "0"	; if mul_dn.drug_name exists for this CNUM but ocs.mnemonic does not match mul_dn.drug_name, return "0"
	endif	
	else "-"	; if neither mul_manf_dn.manufacturer_ordered_drug_name nor mul_dn.drug_name exist for this CNUM, return "-"
	endif	
	, eService_logged = "=IF(ISNA(VLOOKUP(INDIRECT(CHAR(36)&CHAR(72)&ROW()),'Multum SRs'!$B:$B,1,FALSE))=TRUE,0,1)"	
	, synonym_active = ocs.active_ind	
	, synonym_hide = ocs.hide_flag	
	, multum_hide = if (ocs.cki = "MUL*" and ocs.cki = mocl.synonym_cki) cnvtstring (mocl.hide_ind)	; if CNUM is populated and a valid Multum CNUM, return Multum hide
	else "-"	; if CNUM is not populated or valid, return "-"
	endif	
	, hide_match = if (ocs.active_ind = 1 and ocs.cki = "MUL*" and ocs.cki = mocl.synonym_cki)	; if CNUM is populated and a valid Multum CNUM
	if (mocl.hide_ind = ocs.hide_flag) "1"	; if ocs.hide matches Multum hide, return "1"
	else "0"	; if ocs.hide does not match Multum hide, return "0"
	endif	
	else "-"	; if CNUM is not populated or valid, return "-"
	endif	
	, synonym_id = ocs.synonym_id	
	, oc_rank = dense_rank() over (partition by 0	; no logical database field partition
	order by cv_cat.display_key	
	, cv_act.display_key	
	, cv_sub.display_key	
	, cnvtupper(oc.primary_mnemonic)	
	)	
		
from	order_catalog  oc	
	, (left join code_value cv_cat on cv_cat.code_value = oc.catalog_type_cd)	
	, (left join code_value cv_act on cv_act.code_value = oc.activity_type_cd)	
	, (left join code_value cv_sub on cv_sub.code_value = oc.activity_subtype_cd)	
	, (left join br_name_value b_dnum_ig on cnvtint(b_dnum_ig.br_value) = oc.catalog_cd	
	and b_dnum_ig.br_nv_key1 = "MLTM_IGN_DNUM"	
	)	
 	, (left join order_catalog_synonym ocs on ocs.catalog_cd =  oc.catalog_cd	
	and ocs.mnemonic_type_cd not in (2584)	; exclude RxMnemonics
	)	
	, (left join br_name_value b_cnum_ig on cnvtint(b_cnum_ig.br_value) = ocs.synonym_id	
	and b_cnum_ig.br_nv_key1 = "MLTM_IGN_CNUM"	
	)	
	, (left join mltm_drug_name mul_dn on concat("MUL.ORD-SYN!",cnvtstring(mul_dn.drug_synonym_id)) = ocs.cki)	
	, (left join mltm_manufact_drug_name mul_manf_dn on concat("MUL.ORD-SYN!",cnvtstring(mul_manf_dn.drug_synonym_id)) = ocs.cki)	
	, (left join mltm_drug_name_map mul_dn_map on concat("MUL.ORD-SYN!",cnvtstring(mul_dn_map.drug_synonym_id)) = ocs.cki	
	and mul_dn_map.function_id != 29	; exclude 'Brand Drug Name' mappings as these create duplicates
	)	
 	, (left join mltm_order_catalog_load mocl on mocl.synonym_cki = ocs.cki)	
		
plan	oc 	
where	oc.catalog_type_cd = 2516	; code value for 'Pharmacy' from code set 6000
and	oc.orderable_type_flag not in (6,8)	; exclude care sets and IV sets
join	cv_cat	
join	cv_act	
join	cv_sub	
join	b_dnum_ig	
join	ocs 	
join	b_cnum_ig	
join	mul_dn	
join	mul_manf_dn	
join	mul_dn_map	
join	mocl	
		
order by		
	cv_cat.display_key	
	, cv_act.display_key	
	, cv_sub.display_key	
	, cnvtupper(oc.primary_mnemonic)	
	, evaluate(ocs.mnemonic_type_cd	; synonym_type custom list with "Primary" first, as per DCP tools.
	, 2583, 01	; "Primary"
	, 2579, 02	; "Ancillary"
	, 2580, 03	; "Brand Name"
	, 614542, 04	; "C - Dispensable Drug Names"
	, 2581, 05	; "Direct Care Provider"
	, 614543, 06	; "E - IV Fluids and Nicknames"
	, 2582, 07	; "Generic Name"
	, 614544, 08	; "M - Generic Miscellaneous Products"
	, 614545, 09	; "N - Trade Miscellaneous Products"
	, 614546, 10	; "Outreach"
	, 614547, 11	; "PathLink"
	, 2584, 12	; "Rx Mnemonic"
	, 2585, 13	; "Surgery Med"
	, 614548, 14	; "Y - Generic Products"
	, 614549, 15	; "Z - Trade Products"
	)	
	, ocs.mnemonic_key_cap	
	, ocs.synonym_id	; in case 'select distinct' is used
	, 0	; in case 'select distinct' is used
		
with	time = 60	
"""
copy_code_block_to_clipboard(code_block)

<IPython.core.display.Javascript object>

In [29]:
#COPY Current Multum Extract Script
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
# Whatever you put in the code block below will copy
code_block = """select	; current Multum	
	mocl_DNUM = mocl.catalog_cki	
	, mocl_primary_mnemonic = mocl.description	
	, mocl_synonym_type = mocl.mnemonic_type	
	, mocl_synonym_mnemonic = mocl.mnemonic	
	, mocl_active = mocl.active_ind	
	, mocl_hide = mocl.hide_ind	
	, mocl_CNUM = mocl.synonym_cki	
	, synonym_built = if (ocs.cki = mocl.synonym_cki) 1	; use curdomain, if possible for column header: "synonym_exists_in_[curdomain]"
	else 0	
	endif	
	, ignore_build_table = if (b.br_name_value_id > 0) 1	
	else 0	
	endif	
	, ignore_build_discrepancy = if (b.br_name_value_id = 0) "-"	
	elseif (b.br_name_value_id > 0 and ocs.cki = mocl.synonym_cki) "0"	
	else "1"	
	endif	
	  	
from 	mltm_order_catalog_load mocl	
	, (left join order_catalog_synonym ocs on ocs.cki = mocl.synonym_cki)	
	, (left join br_name_value b on b.br_value = mocl.synonym_cki	
	and b.br_nv_key1 = "MLTM_IGN_CONTENT"	
	)	
		
plan	mocl	
		
join	ocs	
join	b	
		
order by	cnvtupper(mocl.description)	
	, evaluate(mocl.mnemonic_type	; synonym_type custom list with "Primary" first
	, "Primary", 1	
	, "Ancillary", 2	
	, "Brand Name", 3	
	, "C - Dispensable Drug Names", 4	
	, "Direct Care Provider", 5	
	, "E - IV Fluids and Nicknames", 6	
	, "Generic Name", 7	
	, "M - Generic Miscellaneous Products", 8	
	, "N - Trade Miscellaneous Products", 9	
	, "Outreach", 10	
	, "PathLink", 11	
	, "Rx Mnemonic", 12	
	, "Surgery Med", 13	
	, "Y - Generic Products", 14	
	, "Z - Trade Products", 15	
	)	
	, mocl.mnemonic_key_cap	
		
with	time = 60	
"""
copy_code_block_to_clipboard(code_block)

<IPython.core.display.Javascript object>

In [32]:
# COPY obsolete Multum extract script
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
code_block = """select distinct	; obsolete Multum	; select distinct required as mmdc_map has multiple rows for each mltmdn.drug_synonym_id
	multum_catalog_cki = mltmmdc.drug_identifier	
	, multum_primary_mnemonic = mltmdn_prim.drug_name	
	, multum_synonym_type = evaluate(mmdc_map.function_id	
	, 16, "Primary"	
	, 29, "Brand Drug Name"	
	, 17, "Brand Name"	
	, 26, "C - Dispensable Drug Names"	
	, 19, "Drug Nickname"	
	, 62, "E - IV Fluids and Nicknames"	
	, 61, "IV Fluid"	
	, 59, "M - Generic Miscellaneous Products/Y - Generic Products"	
	, 64, "Multivitamin Name"	
	, 60, "N - Trade Miscellaneous Products/Z - Trade Products"	
	, 0, "-synonym not found on mltmmap table-"	
	)	
	, multum_synonym_mnemonic = mltmdn.drug_name	
	, multum_synonym_cki = concat("MUL.ORD-SYN!",cnvtstring(mltmdn.drug_synonym_id))	
	, current_multum_indicator = evaluate(mltmdn.is_obsolete,"T",0,"F",1)	
	, mltmdn.drug_synonym_id	
	, synonym_active_in_domain = if (ocs.active_ind = 1 and ocs.cki = concat("MUL.ORD-SYN!",cnvtstring(mltmdn.drug_synonym_id))) "1"	
	else "0"	
	endif	
		
from 		
	mltm_drug_name mltmdn	
	, (left join mltm_mmdc_name_map mmdc_map on mmdc_map.drug_synonym_id = mltmdn.drug_synonym_id)	
	, (left join mltm_ndc_main_drug_code mltmmdc on mltmmdc.main_multum_drug_code = mmdc_map.main_multum_drug_code)	
	, (left join mltm_drug_id mltmdid on mltmdid.drug_identifier = mltmmdc.drug_identifier)	
	, (left join mltm_drug_name mltmdn_prim on mltmdn_prim.drug_synonym_id = mltmdid.drug_synonym_id)	
	, (left join order_catalog_synonym ocs on ocs.cki = concat("MUL.ORD-SYN!",cnvtstring(mltmdn.drug_synonym_id)))	
		
plan	mltmdn	
where	mltmdn.is_obsolete = "T"	; obsolete Multum synonyms only
		
join	mmdc_map	
join	mltmmdc	
join	mltmdid	
join	mltmdn_prim	
join	ocs	
		
order by 		
	cnvtupper(mltmdn_prim.drug_name)	; Multum Primary Mnemonic
	, evaluate(mmdc_map.function_id	; synonym_type custom list with "Primary" first, as per DCP tools.
	, 16, 1	; Primary
	, 29, 2	; Brand Drug Name
	, 17, 3	; Brand Name
	, 26, 4	; C - Dispensable Drug Names
	, 19, 5	; Drug Nickname
	, 62, 6	; E - IV Fluids and Nicknames
	, 61, 7	; IV Fluid
	, 59, 8	; M - Generic Miscellaneous Products/Y - Generic Products
	, 64, 9	; Multivitamin Name
	, 60, 10	; N - Trade Miscellaneous Products/Z - Trade Products
	)	
	, cnvtupper(mltmdn.drug_name)	; Multum Synonym Mnemonic
"""
copy_code_block_to_clipboard(code_block)

<IPython.core.display.Javascript object>

In [47]:
# Read Spreadsheet - 'p2031 syn-mul compare extract script.csv'

spreadsheet_filename = "p2031 syn-mul compare extract script.csv"
file_encoding = 'Windows-1252'

import pandas as pd
#read_file = pd.read_csv (target_location, engine = 'python', dtype = 'str', delimiter= '|')
df_syn_mul = pd.read_csv (spreadsheet_filename, dtype = 'str', encoding=file_encoding)
print (df_syn_mul.head(1))

  PRIMARY_DNUM MULTUM_DNUM DNUM_MATCH  IGNORE_DNUM_MAPPING  \
0       IGNORE           -          -                    0   

         PRIMARY_MNEMONIC  PRIMARY_CATALOG_CD SYNONYM_CNUM  \
0  *** Blister Pack C & C        103045952.00          NaN   

   IGNORE_CNUM_MAPPING      CNUM_COUNT MULTUM_SYN_CKI_STATUS  ...  \
0                    0         3203.00                     -  ...   

   DUPLICATE_MNEMONIC_COUNT MULTUM_MNEMONIC MNEMONIC_MATCH  \
0                      1.00               -              -   

                                     ESERVICE_LOGGED  SYNONYM_ACTIVE  \
0  =IF(ISNA(VLOOKUP(INDIRECT(CHAR(36)&CHAR(72)&RO...               1   

   SYNONYM_HIDE MULTUM_HIDE HIDE_MATCH      SYNONYM_ID         OC_RANK  
0             0           -          -    103045955.00            1.00  

[1 rows x 24 columns]


# Create Update code to switch obsolete CNUMS to new CNUMS
This step deals with situations like, for example, the product is changed in some way. When this happens multum simply creates a whole new cnum. To maintain the current catalog build and link all the functionality that comes with Multum, we must overwrite the old CNUM with the new CNUM.

This step generates code that updates CNUMS where:
- the current CNUM is obsolete (column 'MULTUM_SYN_CKI_STATUS' is 'obsolete')
- there is an exact match between the catalog column 'SYNONYM_MNEMONIC' in P2031 and current multum column 'MOCL_SYNONYM_MNEMONIC'

In [56]:
# filter for obsolete items
#region
import pandas as pd
#read "p2031 syn-mul compare extract script.csv"
spreadsheet_filename = "p2031 syn-mul compare extract script.csv"
file_encoding = 'Windows-1252'
df_syn_mul = pd.read_csv (spreadsheet_filename, dtype = 'str', encoding=file_encoding)
print('P2031 syn-mul: ', df_syn_mul.shape)
# filter obsoletes in "p2031 syn-mul compare extract script.csv"
df_syn_mul_filtered = df_syn_mul[df_syn_mul['MULTUM_SYN_CKI_STATUS'] == 'obsolete']
print('P2031 syn-mul obsoletes: ', df_syn_mul_filtered.shape)
#obsolete items to excel file
#df_syn_mul_filtered.to_excel("df_syn_mul_filtered.xlsx", index=False) # this line will export the obsolete cnums to a spreadsheet
#endregion

# Read Spreadsheet - 'current multum.csv'
#region
spreadsheet_filename = "current multum.csv"
#file_encoding = 'Windows-1252'
import pandas as pd
#read_file = pd.read_csv (target_location, engine = 'python', dtype = 'str', delimiter= '|')
df_current_multum = pd.read_csv (spreadsheet_filename, dtype = 'str' ) #encoding=file_encoding
print('Current Multum: ', df_current_multum.shape)
#endregion

# Read Spreadsheet - 'p2031 syn-mul compare extract script.csv'
#region
file_encoding = 'Windows-1252'
import pandas as pd
df_syn_mul = pd.read_csv (spreadsheet_filename, dtype = 'str', encoding=file_encoding)
#endregion

P2031 syn-mul:  (23293, 24)
P2031 syn-mul obsoletes:  (57, 24)
Current Multum:  (36322, 10)


In [65]:
# Generate CCL Code for Updating obsolete CNUMs

ccl_code_start ="""drop program WHS_bulk_updates go
create program WHS_bulk_updates
prompt
"Output to File/Printer/MINE" = "MINE"
with OUTDEV
;insert update scripts below---------------------------\n"""

ccl_code_loop ='; CNUM UPDATE (non-domain-specific)\n\
update into order_catalog_synonym ocs\n\
set ocs.cki = "MUL.ORD-SYN!000000"\n\
, ocs.concept_cki = "MULTUM!000000"\n\
, ocs.updt_dt_tm = cnvtdatetime(curdate,curtime3)\n\
, ocs.updt_id = reqinfo->updt_id\n\
, ocs.updt_cnt = ocs.updt_cnt + 1\n\
where ocs.mnemonic = "SWAP_ME_WITH_SYNONYN_MNEMONIC"\n'

ccl_code_end = ";------------------------------------------------------\nend\ngo"

ccl_code_middle = ""

for SYNONYM_MNEMONIC in df_syn_mul_filtered['SYNONYM_MNEMONIC']:
    if SYNONYM_MNEMONIC in df_current_multum['MOCL_SYNONYM_MNEMONIC'].values:
        temp_string = ccl_code_loop.replace("SWAP_ME_WITH_SYNONYN_MNEMONIC", SYNONYM_MNEMONIC)
        cnum_code_string = df_current_multum.loc[df['MOCL_SYNONYM_MNEMONIC'] == SYNONYM_MNEMONIC, 'MOCL_CNUM'].iloc[0]
        temp_string = temp_string.replace("MUL.ORD-SYN!000000", cnum_code_string)
        ccl_code_middle += temp_string + '\n'

ccl_code_final = ccl_code_start + ccl_code_middle + ccl_code_end

print (ccl_code_final)

drop program WHS_bulk_updates go
create program WHS_bulk_updates
prompt
"Output to File/Printer/MINE" = "MINE"
with OUTDEV
;insert update scripts below---------------------------
; CNUM UPDATE (non-domain-specific)
update into order_catalog_synonym ocs
set ocs.cki = "MUL.ORD-SYN!000000"
, ocs.concept_cki = "MULTUM!000000"
, ocs.updt_dt_tm = cnvtdatetime(curdate,curtime3)
, ocs.updt_id = reqinfo->updt_id
, ocs.updt_cnt = ocs.updt_cnt + 1
where ocs.mnemonic = "Addaven"

;------------------------------------------------------
end
go


In [ ]:
# 
from IPython.display import display, Javascript
import clipboard
def copy_code_block_to_clipboard(code_block):
    """
    Copy a code block to the clipboard in a Jupyter Notebook.

    Args:
        code_block (str): The code block to copy.
    """
    # Create the JavaScript code to copy the code block to the clipboard
    js_code = f"""
        var code = `{code_block}`;
        navigator.clipboard.writeText(code);
    """
    # Display the JavaScript code in the notebook
    display(Javascript(js_code))
    # Copy the code to the clipboard using the clipboard module
    clipboard.copy(code_block)
code_block = """
"""
copy_code_block_to_clipboard(code_block)

## Multum 3.7 Missing DNUMS
time: 0.5 seconds
### Set initial variables

In [7]:
#excel_file_path = r'\\whoffice\shared\EMR\BAU\Audit Spreadsheets\Multum audits\Multum Processing 6 - catalogue-Multum_Alignment.xlsx'
excel_file_path = r'C:\storage\data\Multum Processing 6 - catalogue-Multum_Alignment.xlsx'
output_file_path = r'C:\storage\data\dataout.xlsx'
PROD = 'P2031'
NONPROD = 'C2031'
# Below are a list of words that have zero weighting towards the matching algorithm between multum and our catelog
no_match_lst = ['topical', 'nasal']

## Import Libraries and read Multum 6
### Time to execute block below
* over network: 5 mins 15 seconds (approx 320 seconds)
* from local SSD: 17 seconds

In [8]:
# Import Libraries
import pandas as pd
import numpy as np
import openpyxl as pyxl
#import os

# Read Multum 6
m = pd.read_excel(excel_file_path, sheet_name=None)
print('ignore warnings above this line______')

# Show sheet names in Multum 6
print('---EXCEL SHEET NAMES---')
for x in m.keys():
    print(x)
print('-----------------------')

if PROD not in m.keys():
    print('Your itnitial conditions are not correctly set')
else:
    print('You may continue')

ignore warnings above this line______
---EXCEL SHEET NAMES---
C2031
P2031
syn-Mul compare extract script
current Multum
current Multum extract script
Multum build exceptions
obsolete Multum
obsolete Multum extract script
Multum SRs
-----------------------
You may continue


## 2.7.4.
time: 0.1 seconds
The code block below does the equivalent of:

"On the selected environment’s worksheet, filter column F [PRIMARY_MNEMONIC] to show only light yellow coloured cells. These are active primaries that do not have a DNUM."

In [9]:
# Filter for active primaries that do not have a DNUM. and put in table p_1
# Equivalent to filtering for 'yellow' on the 'PRIMARY_MNEMONIC' column (in the excel tool)
prod_data = m[PROD].fillna('', inplace = False) # blank cells are literally blank strings '' now.
prod_data = prod_data.astype(str)
prod_data.columns = prod_data.columns.str.replace(' ','_') # Changing column names to get rid of spaces
prod_data.columns = prod_data.columns.str.replace("'",'')
# prod_active_primaries = prod.query(" SYNONYM_TYPE == 'Primary' & SYNONYM_ACTIVE == 1 & PRIMARY_DNUM == '' ")
p_1 = prod_data
p_1 = p_1.loc[p_1['SYNONYM_TYPE'] == 'Primary']
p_1 = p_1.loc[p_1['SYNONYM_ACTIVE'] == '1']
p_1 = p_1.loc[p_1['PRIMARY_DNUM'] == '']

## Missing DNUMS
time: 0.6 seconds

## The block below will get all the 'current Multum' DNUMS that are not associated with items in our prod catalogue

In [10]:

# Rename the table, make all data in the cells strings and rename columns
m_1 = m['current Multum'].astype(str)
m_1.columns = m_1.columns.str.replace(' ','_') # Changing column names to get rid of spaces
m_1.columns = m_1.columns.str.replace("'",'') # Changing column names to get rid of thes
# Filtering Dataframe
# the stuff below does the same as this formula in the current Multum sheet. Every line filters the data some more
# Formula: =AND($B1>0,$I1=0,$J1=0,$M1<>1,$P1<>1,$S1<>1,$V1<>1,$Y1<>1,$AB1<>1,$AE1<>1)
m_1 = m_1.loc[m_1['MOCL_SYNONYM_TYPE'] == 'Primary']
m_1 = m_1.loc[m_1['MOCL_DNUM'].str.startswith('MUL')]
m_1 = m_1.loc[m_1['SYNONYM_BUILT']  == '0' ]
m_1 = m_1.loc[m_1['generic_manufacturer_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['multi-ingredient_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['volume_dosed_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['flu_vax_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['non-substitutable_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['PBS_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['miscellaneous_do_not_build']  != '1' ]
m_1.reset_index(drop=True, inplace=True) # Reindex the data (optional)

# Rename the table, make all data in the cells strings and rename columns
m_2 = m['current Multum'].astype(str)
m_2.columns = m_2.columns.str.replace(' ','_') # Changing column names to get rid of spaces
m_2.columns = m_2.columns.str.replace("'",'')
# Filtering Dataframe with the second conditional formula
# the stuff below does the same as this formula in the current Multum sheet. Every line filters the data some more
# Formula: =AND($I1=0,$AF1=1)
m_2 = m_2.loc[m_2['SYNONYM_BUILT']  == '0' ]
m_2 = m_2.loc[m_2['miscellaneous_do_build']  == '1' ]
m_2.reset_index(drop=True, inplace=True) # Reindex the data (optional)

# Merge m_1 and m_2
free_mul_primaries = pd.concat([m_1, m_2], ignore_index=True, sort=True)
# Get rid of duplicates
free_mul_primaries.drop_duplicates(inplace = True) 
free_mul_primaries.reset_index(drop=True, inplace=True) # Reindex the data (optional)

## Generate Excel File so user can choose associations.
time: 0.5 seconds

In [11]:
# Unmatched Primaries in Prod
p_2 = p_1.filter(['PRIMARY_MNEMONIC'])
p_2.reset_index(drop=True, inplace=True)

# Create a df for Multum - Catelog matching spreadsheet
column1 = 'PRIMARY_MNEMONIC' #cerner
column2 = 'MOCL_PRIMARY_MNEMONIC' #multum
column3 = 'MOCL_DNUM' # Multum
column4 = 'MOCL_CNUM' # Multum
#df = pd.DataFrame(columns=[column1, column2])
df = pd.DataFrame(columns = [column1,column2, column3, column4])

# Array of all unmatched Multum Primaries
mul_p = free_mul_primaries.filter(items = [column2, column3, column4])

# Matching Algorithm
for index, row in p_2.iterrows():
    cernername = row[column1]
    current_key_words = cernername.split(' ')
    for i, r in mul_p.iterrows():
        multumname = r[column2]
        words_multumname = multumname.split(' ')
        # Remove the words that should have no impact on matching such as 'topical'
        words_y = [i for i in words_multumname if i not in no_match_lst]
        # Checking if any unallocated catelog primaries match with any unallocated multum primaries and add to table
        if bool(set(current_key_words) & set(words_y)):
            #make a mini table
            data = {column1: [cernername], column2: [multumname], column3: [r[column3]], column4: [r[column4]]}  
            df_temp = pd.DataFrame(data)
            # append to main table
            df = pd.concat([df, df_temp], ignore_index=True)

# Add match column
column0 = 'MATCH? "x"'
df[column0]=''
# reorder columns
column_order = [column0,column1,column2,column3,column4]
df = df[column_order]

# Replace duplicates in the PRIMARY_MNEMONIC column with blanks, keep first
prev_row = 'x'
for index, row in df.iterrows():
    if row['PRIMARY_MNEMONIC'] == prev_row:
        df.loc[index, 'PRIMARY_MNEMONIC'] = ''
    prev_row = row['PRIMARY_MNEMONIC']

df.to_excel(output_file_path)

## Read the excel sheet

In [12]:
df_back_in = pd.read_excel(output_file_path, sheet_name=None)